# Validating Nash Equilibriums Via Simulations

_by Colin Manko_

In an effort to validate and test possible improvements to core poker artificial intelligence algorithms, I have designed the following methodology.

### Goals

- Validate that MCCFR offline learning strategy is approximating a Nash equilibrium
- More generally, create a methodology that allows for rigorously testing changes made to the core AI algorithms

#### Prerequisites

- Need two test bot implementation strategies ($\beta{1}$ and $\beta{2}$) that we would like to compare
- Need a human tester ($H_{0}$) as a quasi control. The human tester should not have access to any underlying strategies from the test bots or simulated Nash

#### Step 1:  Randomly Generate Test Game

Given a set of $N$ game tree nodes (this is the entire game tree, as given by infoset, $I$), randomly generate $x$ test nodes without preplacement and with equal probability. Call the set of test nodes $U$.

As a side note, we will account for probability of reach ($p(h)$) in another step. Equal probability across nodes allows us to find patterns across nodes where our agent underperforms. We will adjust the expected value at $I$, ($v^{\sigma}(I)$), by $p(h)$.
- **How to**: For Limit Texas Hold'em, the number of action sequences ($N$), is small enough that they can be found computationally rather than analytically. We can run *all_action_sequences.py* in the *size_of_problem* directory to generate this list. 
- _Something like 15-20 hours and less than 4GB??_
- Generate $x$ integers to be indices and select them from the *all_action_sequences.py* output
- Once $x$ action sequences are generated, randomly generate $x$ public card combos, based on the betting stage of the test node, $u$, as well as one pair of private hole cards to be used by $\beta{1}$, $\beta{2}$ and $H_0$. They will only get that hand at $u$. 

#### Step 2: Prepare Realtime Search for Finding Nash Equilibrium

For each test node, $u$, in $U$, use realtime search to compute the Nash Equilibrium ($\sigma^*$) by constraining the search algorithm to start at $u$, where $u$ is equivalent to $I$ in regard to action sequence, but does not have any set hand for the traversing player ($p_i$).

Use a pooled strategy between $\beta{1}$ and $\beta{2}$ to estimate $p(h)$ without bias:

For hand in possible combinations of real hands:
<br>
&nbsp;&nbsp;&nbsp;&nbsp;For idx, $a$ in action sequence at $u$:
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;if idx == 0:
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$p(h)_\beta{1}$ = $\beta{1}$[$I$][$a$]
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$p(h)_\beta{2}$ = $\beta{2}$[$I$][$a$]
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$p(h)_\beta{1}$ *= $\beta{1}$[$I$][$a$]
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$p(h)_\beta{2}$ *= $\beta{1}$[$I$][$a$]
<br>
&nbsp;&nbsp;&nbsp;&nbsp;p(h)[rs] = ($p(h)_\beta{1}$ + $p(h)_\beta{2}$)/2

The root node of the realtime search algorithm is replaced with a chance node that represents each possible node in the public state $G$ [[Brown, Sandholm, Amos]](https://papers.nips.cc/paper/7993-depth-limited-solving-for-imperfect-information-games.pdf). From the above psuedo-code, this deal can be generated as: 

Generate deal order
<br>
For $i$ in $P_i$ deal order:
<br>
&nbsp;&nbsp;&nbsp;&nbsp;Generate hand for player based on normalized $p(h)[rs]$ if available, else try again

_The "if available, else try again" part could be made better_

**Other important features of the _"Nash Bot"_ real time search..**:
- The _"Nash bot"_ is the master of this node. In order to reach full convergence, from the normal MCCFR algorithm, we must remove the sampling of actions for opponents.
- For ease, the real time search should not use leaf nodes, but should search to the end of the game tree, where either a terminal node or a shown down is entered. In this way, we can get a truer sense of the expected value.

A Nash Equilibrium is found if the change in strategy on each iteration drops below some threshold $t$ for the real hand we are testing for. Charting probabilities for each action in $u$ over time for the randomly generated real hand to test should show a convergence over time.

_One main benefit of using this real time search to validate CFR is this search will need to be developed anyway._

#### Step 3: Test and Measure Success

**For the test bots:**
For each $u$ in $U$, play each test strategy ($\beta{1}$ and $\beta{2}$) against the _"Nash bot"_ for $r$ number of simulations. The _"Nash bot"_ should be dealt available hands from the distribution of probabilities as determined by $p(h)[rs]$ in the pseudo-code above. Both the test bots and the human tester will be dealt the same hand in each simulation of game play on $u$, as randomly generated in step 1. 

If $\beta{1}$ or $\beta{2}$ has converged to a Nash equilibrium, then we should expect $v^\sigma$ to be equal to 0 for our test bot, assuming that _"Nash bot"_ has converged to a Nash equilibrium itself. $v^{\sigma^*}(u)$ and $v^{\sigma}(u)$ are the estimated payouts for the _"Nash bot"_ opponents and the "hero" (test bots or human), respectively.

**For the human tester:**
We can simply create a contrived game. Based on the normalized probability of reach for $u$, $\bar{p(h)}$, we can randomly generate which $u$ the human player is entered into, however they will always have the same hand upon entering $u$ and their opponents hands will vary based on $p(h)[rs]$.

The test metric is as follows, after $p(h)$ has been normalized for space $U$, $\bar{p(h)}$:
$$\sum_{i=1}^{x}(v^{\sigma}(u_i)-v^{\sigma^*}(u_{-i}))\times{\bar{p(h)}}$$

The value closest to 0 (summing no testing agent goes over 0) will have best approximated the Nash equilibrium. Additionaly, $H_0$ can be used as a quasi-control, to validate that the bot is beating a human.

The above metric also has some degree of simulation error. For each simulation in $r$ simulations, we create a distribution of values that has a standard deviation and follows the normal distribution. 

Along with calculating the expected payout per simulation, $u^{\sigma}(u_i)-u^{\sigma^*}(u_{-i})$, we can also calculate $\sigma$ for this distribution in order to describe a confidence interval around the test metric. 

Finally, a simple difference of means can be done between each test bot to decipher a winner and if that winner had a statistically significant edge. We can then study each $u$ in $U$ to find patterns in which nodes the espspective bots did not do well with.